In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import STL
import matplotlib.font_manager as fm
import seaborn as sns
import math
import re
from sklearn.preprocessing import LabelEncoder

In [2]:
# 파일 읽기
df23 = pd.read_csv("C:/Users/etbae/Downloads/mainC/train_subway23.csv")

# 역 이름 인코딩
le = LabelEncoder()
df23['train_subway23.station_name_encoded'] = le.fit_transform(df23['train_subway23.station_name'])

# 방향: 바이너리 인코딩
df23['train_subway23.direction_bin'] = df23['train_subway23.direction'].map({'내선': 0, '상선': 0, '외선': 1, '하선': 1})

# 날짜 분할
df23['train_subway23.datetime'] = pd.to_datetime(df23['train_subway23.tm'], format='%Y%m%d%H')

# 수치형 열만 선택
numeric_cols = df23.select_dtypes(include=[np.number]).columns
# 각 열에 대해 이상치 처리: -99 이하인 값은 NaN으로
for col in numeric_cols:
    df23[col] = df23[col].map(lambda x: np.nan if isinstance(x, (int, float)) and x <= -99 else x)
# 추가 처리: wd 값이 -9.9이면 NaN
df23['train_subway23.wd'] = df23['train_subway23.wd'].map(lambda x: np.nan if x == -9.9 else x)


##### * 보간 방식 선택: 사용하지 않는 방식을 주석처리 *

In [3]:
# 1. 정렬 --------------------------------------------------------------
df23 = df23.sort_values("train_subway23.datetime")

# 2. 대상 컬럼 ---------------------------------------------------------
cols_to_clean = ["train_subway23.ta", "train_subway23.ta_chi", "train_subway23.ws"]

# 3. datetime을 인덱스로 설정 ------------------------------------------
df23 = df23.set_index("train_subway23.datetime")

# 4. 보간 방식 선택 (주석처리로 둘 중 하나 선택) ------------------------
# --- 방법 1: 시간 기준 선형 보간 (index 간격 활용) ---
df23[cols_to_clean] = df23[cols_to_clean].interpolate(method="time", limit_direction="both")

# --- 방법 2: 3차 스플라인 보간 ---
# df23[cols_to_clean] = df23[cols_to_clean].interpolate(method="spline", order=3, limit_direction="both")

# 5. 인덱스 복구 -------------------------------------------------------
df23 = df23.reset_index()

# 6. 확인 --------------------------------------------------------------
print(df23[cols_to_clean].isna().sum())  # 남은 NaN 개수


train_subway23.ta        0
train_subway23.ta_chi    0
train_subway23.ws        0
dtype: int64


In [4]:
# 1. 정렬 --------------------------------------------------------------
df23 = df23.sort_values("train_subway23.datetime")

# 2. 대상 컬럼 ---------------------------------------------------------
cols_to_clean = ["train_subway23.hm"]

# 3. datetime을 인덱스로 설정 ------------------------------------------
df23 = df23.set_index("train_subway23.datetime")

# 4. 보간 방식 선택 (주석처리로 둘 중 하나 선택) ------------------------
# --- 방법 1: 시간 기준 선형 보간 ---
df23[cols_to_clean] = df23[cols_to_clean].interpolate(method="time", limit_direction="both")

# --- 방법 2: 3차 스플라인 보간 ---
# df23[cols_to_clean] = df23[cols_to_clean].interpolate(method="spline", order=3, limit_direction="both")

# 5. 인덱스 복구 -------------------------------------------------------
df23 = df23.reset_index()

# 6. 확인 --------------------------------------------------------------
print(df23[cols_to_clean].isna().sum())  # 남은 NaN 개수


train_subway23.hm    0
dtype: int64


In [5]:
# 1. 대상 컬럼 ---------------------------------------------------------
col = 'train_subway23.rn_hr1'

# 2. 정렬 -------------------------------------------------------------
df23 = df23.sort_values("train_subway23.datetime")

# 3. 인덱스 설정 ------------------------------------------------------
df23 = df23.set_index('train_subway23.datetime')

# 4. 결측 여부 및 마스크 정의 ------------------------------------------
isna = df23[col].isna()
left0 = df23[col].shift(1).fillna(0) == 0
right0 = df23[col].shift(-1).fillna(0) == 0
zero_mask = isna & (left0 | right0)
interpolate_mask = isna & (~zero_mask)

# 5. 결측값 처리 -------------------------------------------------------
# (1) 양쪽이 0인 경우 → 0으로 대체
df23.loc[zero_mask, col] = 0

# (2) 나머지 → 보간
# 보간 방식 선택 (주석처리로 둘 중 하나 선택) -------------------------
# --- 방법 1: 시간기반 보간 ---
df23.loc[interpolate_mask, col] = df23[col].interpolate(method='time', limit_direction='both')[interpolate_mask]

# --- 방법 2: 스플라인 보간 ---
# df23.loc[interpolate_mask, col] = df23[col].interpolate(method='spline', order=3, limit_direction='both')[interpolate_mask]

# 6. 인덱스 복구 ------------------------------------------------------
df23 = df23.reset_index()

# 7. 결과 확인 --------------------------------------------------------
missing_count = df23[col].isna().sum()
print(f"[{col}] 결측치 남은 개수:", missing_count)


[train_subway23.rn_hr1] 결측치 남은 개수: 0


In [6]:
import pandas as pd
import numpy as np

# 1. 보간 대상 컬럼 및 시간정보 생성 ------------------------------
col = 'train_subway23.si'

df23['month'] = df23['train_subway23.datetime'].dt.month
df23['hour'] = df23['train_subway23.datetime'].dt.hour

# 2. 월-시 기준 NaN 비율 계산 후 밤 시간대 추정 ------------------
threshold = 0.9  # 밤으로 간주할 NaN 비율 기준

si_null_ratio = (
    df23.groupby(['month', 'hour'])[col]
    .apply(lambda x: x.isna().mean())
    .reset_index(name='null_ratio')
)

# 월별 밤 시간대 목록 추출
night_hours_by_month = (
    si_null_ratio[si_null_ratio['null_ratio'] >= threshold]
    .groupby('month')['hour']
    .agg(list)
    .to_dict()
)

# 3. 낮/밤 플래그 생성 -------------------------------------------
def is_daytime(row):
    return row['hour'] not in night_hours_by_month.get(row['month'], [])

df23['is_daytime'] = df23.apply(is_daytime, axis=1)

# 4. 밤 시간대 NaN → 0으로 대체 ----------------------------------
df23.loc[~df23['is_daytime'] & df23[col].isna(), col] = 0

# 5. 낮 시간대 NaN → 전체 보간 ------------------------------------
# 정렬 및 인덱스 설정
df23 = df23.sort_values("train_subway23.datetime")
df23 = df23.set_index('train_subway23.datetime')

# 낮 시간대 마스크
day_mask = df23['is_daytime'] & df23[col].isna()

# 보간 방식 선택 (주석처리로 둘 중 하나 선택) ----------------------
# --- 방법 1: 시간 기반 보간 ---
df23.loc[day_mask, col] = df23[col].interpolate(method='time', limit_direction='both')[day_mask]

# --- 방법 2: 3차 스플라인 보간 ---
# df23.loc[day_mask, col] = df23[col].interpolate(method='spline', order=3, limit_direction='both')[day_mask]

# 인덱스 복구
df23 = df23.reset_index()

# 6. 남은 결측치 확인 --------------------------------------------
missing_count = df23[col].isna().sum()
print(f"[{col}] 남은 결측치 개수:", missing_count)


[train_subway23.si] 남은 결측치 개수: 0


In [7]:
# 1. 날짜 컬럼 생성 --------------------------------------------------
# (기존에 있으면 생략 가능)
df23['date'] = df23['train_subway23.datetime'].dt.date

# 2. 대상 컬럼 지정 -------------------------------------------------
rn_day_col = 'train_subway23.rn_day'
rn_hr1_col = 'train_subway23.rn_hr1'

# 3. 일별 rn_hr1 합계 계산 ------------------------------------------
# groupby는 필수 (일 단위 합산이 목적이므로)
sum_rn_hr1 = (
    df23.groupby(['train_subway23.station_name_encoded', 'train_subway23.station_number', 'date'])[rn_hr1_col]
    .sum()
    .reset_index()
    .rename(columns={rn_hr1_col: 'sum_rn_hr1'})
)

# 4. 원본 데이터에 일별 합계 merge ----------------------------------
df23 = df23.merge(
    sum_rn_hr1,
    on=['train_subway23.station_name_encoded', 'train_subway23.station_number', 'date'],
    how='left'
)

# 5. rn_day 결측치 대체 --------------------------------------------
# rn_day가 결측이면 sum_rn_hr1 값으로 대체
mask = df23[rn_day_col].isna()
df23.loc[mask, rn_day_col] = df23.loc[mask, 'sum_rn_hr1']

# 6. 결과 확인 ------------------------------------------------------
missing_count = df23[rn_day_col].isna().sum()
print(f"[{rn_day_col}] 남은 결측치 개수:", missing_count)


[train_subway23.rn_day] 남은 결측치 개수: 0


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import xgboost as xgb
import lightgbm as lgb

df23['day'] = df23['train_subway23.datetime'].dt.day

# --- 사용자 지정 입력 변수와 타겟 -------------------------------
target_col = "train_subway23.congestion"
feature_cols = ['train_subway23.line', 'train_subway23.ta', 'train_subway23.ws', 'train_subway23.rn_hr1', 'train_subway23.hm', 'train_subway23.si',
                'train_subway23.ta_chi', 'train_subway23.station_name_encoded', 'train_subway23.direction_bin', 'month', 'hour', 'day']  # 원하는 변수만 넣기

# --- 데이터 전처리 ----------------------------------------------
df_model = df23[feature_cols + [target_col]].dropna()
X = df_model[feature_cols]
y = df_model[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# --- 모델 정의 ---------------------------------------------------
models = {
    "LightGBM": lgb.LGBMRegressor(),
    "XGBoost": xgb.XGBRegressor(),
    "HistGradientBoosting": HistGradientBoostingRegressor(),
    "RandomForest": RandomForestRegressor()
}

# --- 결과 저장 ---------------------------------------------------
results = []

for name, model in models.items():
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    
    r2 = r2_score(y_test, y_pred)
    rmse = root_mean_squared_error(y_test, y_pred)

    results.append({
        "Model": name,
        "R²": round(r2, 4),
        "RMSE": round(rmse, 4)
    })

# --- 결과 출력 ----------------------------------------------------
results_df = pd.DataFrame(results)
print("모델 성능 비교:")
print(results_df.to_string(index=False))


C:\Users\etbae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] 지정된 파일을 찾을 수 없습니다
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Users\etbae\anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\etbae\anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\etbae\anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\etbae\anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
 

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.017521 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1576
[LightGBM] [Info] Number of data points in the train set: 4416784, number of used features: 12
[LightGBM] [Info] Start training from score 25.032588
모델 성능 비교:
               Model     R²    RMSE
            LightGBM 0.4617 15.7247
             XGBoost 0.4932 15.2584
HistGradientBoosting 0.4704 15.5968
        RandomForest 0.7427 10.8716


In [ ]:
# csv 파일을 저장
df23.to_csv("C:/Users/etbae/Downloads/train_subway23_nogroupby_time.csv", index=False, encoding='utf-8-sig')

In [11]:
# 요일, 시간 파생
df23['weekday'] = df23['train_subway23.datetime'].dt.weekday  # 0=월, ..., 6=일
df23['hour'] = df23['train_subway23.datetime'].dt.hour

# 평일 여부 (월~금 = 1, 토/일 = 0)
df23['train_subway23.is_weekday'] = df23['weekday'].apply(lambda x: 1 if x < 5 else 0)

# 혼잡 시간대 여부 (출근 07–09시, 퇴근 17–19시)
df23['train_subway23.rush_hour'] = df23['hour'].apply(lambda h: 1 if h in [7, 8, 9, 17, 18, 19] else 0)


In [12]:
# csv 파일을 저장
df23.to_csv("C:/Users/etbae/Downloads/train_subway23_nogroupby_time_added.csv", index=False, encoding='utf-8-sig')